In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
def parse_model_config(path):
    """Parses the yolo-v3 layer configuration file and returns module definitions"""
    file = open(path, 'r')
    lines = file.read().split('\n')
    lines = [x for x in lines if x and not x.startswith('#')]
    lines = [x.rstrip().lstrip() for x in lines] # get rid of fringe whitespaces
    module_defs = []
    for line in lines:
        if line.startswith('['): # This marks the start of a new block
            module_defs.append({})
            module_defs[-1]['type'] = line[1:-1].rstrip()
            if module_defs[-1]['type'] == 'convolutional':
                module_defs[-1]['batch_normalize'] = 0
        else:
            key, value = line.split("=")
            value = value.strip()
            module_defs[-1][key.rstrip()] = value.strip()

    return module_defs


def save_model_config(model_defs, path):
    """Saves the yolo-v3 layer configuration file"""
    with open(path, 'w') as writer:
        
        for block in model_defs:
            writer.write('['+ block['type'] +']'+'\n')
            [writer.write(str(k)+'='+str(v)+'\n') for k,v in block.items() if k != 'type']
            writer.write('\n')
    return path

            
def save_data_config(data_config, path):
    """Saves the yolo-v3 data configuration file"""
    with open(path, 'w') as writer:
        [writer.write(str(k)+'='+str(v)+'\n') for k,v in data_config.items()]
    return path


def inject_model_config(dataset, model_config, hyperparams):
    ## TODO: Update filters based on number of classes
    for block in model_config:        
        if block['type'] == 'net':
            block['learning_rate'] = hyperparams['lr']
            block['batch'] = hyperparams['batch']
            block['subdivisions'] = hyperparams['subdivisions']
            block['burn_in'] = len(dataset._images.items())//(hyperparams['gpus'] * hyperparams['batch'])
            block['max_batches'] = len(dataset._images.items())//(hyperparams['gpus'] * hyperparams['batch']) * hyperparams['epochs']
    return model_config


def inject_data_config(dataset, data_config):
    data_config['train'] = os.path.join(dataset.darknet_manifast,'manifast.txt')
    data_config['classes'] = len(dataset.category_names)
    ## TODO: Add Validation Set
    data_config['valid'] = os.path.join(dataset.darknet_manifast,'manifast.txt')
    data_config['names'] = dataset.names_config
    backup_path = os.path.abspath(os.path.join(dataset.output_path, os.pardir, 'backup'))
    os.makedirs(backup_path, exist_ok = True)
    data_config['backup'] = os.path.abspath(os.path.join(dataset.output_path, os.pardir, 'backup'))
    num_gpus = int(dataset.parse_nvidia_smi()['Attached GPUs'])
    data_config['gpus'] = ','.join(str(i) for i in range(num_gpus))
    
    
    return data_config
            

def parse_data_config(path):
    """Parses the data configuration file"""
    options = dict()
    options['gpus'] = '0,1'
    with open(path, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.strip()
        if line == '' or line.startswith('#'):
            continue
        key, value = line.split('=')
        options[key.strip()] = value.strip()
    return options



In [3]:
import os
import yaml
import urllib
from PIL import Image
from enum import Enum
from pycocotools.coco import COCO
import xml.etree.cElementTree as ET
import glob
import argparse
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
import scipy.misc
from skimage import measure   
import random
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import shutil
import pickle
import pandas as pd
from subprocess import Popen,PIPE,STDOUT,call 


from utils import datasets

BDD100K_DIRECTORY = os.path.join('/media/dean/datastore1/datasets/BerkeleyDeepDrive', 'bdd100k')
WORKING_DIRECTORY = '/media/dean/datastore1/datasets/darknet'
DATA_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'data')
COCO_DIRECTORY = os.path.join(DATA_DIRECTORY, 'coco')

TRAINERS_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'trainers')
ANNOTATIONS_FILE = os.path.join(BDD100K_DIRECTORY, 'labels/bdd100k_labels_images_val.json')

BASE_DATA_CONFIG = os.path.join(WORKING_DIRECTORY, 'cfg', 'bdd100k.data')
BASE_MODEL_CONFIG = os.path.join(WORKING_DIRECTORY, 'cfg', 'yolov3-bdd100k.cfg')

In [ ]:
# For Run in Training Runs    
trainers = os.listdir(TRAINERS_DIRECTORY)
all_training_runs = []

for trainer in trainers:
    ## Prepare Dataset ##
    bdd_set = datasets.DataFormatter(annotations_list = ANNOTATIONS_FILE, input_format = datasets.Format.bdd,
                                     #pickle_file = 'BerkeleyDeepDrive_bdd100k_labels_kache_bdd100k_labels_images_train.json.pickle',
                                     output_path = os.path.join(TRAINERS_DIRECTORY,trainer,'data'),
                                     trainer_prefix = 'COCO_val2014_0000', 
                                     s3_bucket = 'kache-scalabel/bdd100k/images/100k/val/')
    bdd_set.export(datasets.Format.darknet)

    print('Initiating Trainer:', os.path.join(TRAINERS_DIRECTORY,trainer))
    # Grab hyperparameters from filename
    tokens = trainer.split('_')
    hyperparams = {'name': tokens[0], 
                   'gpus': int(tokens[1].replace('gpu','')),
                   'lr': float('0.'+tokens[2].replace('lr','')),
                   'batch': int(tokens[3].replace('bat','')),
                   'subdivisions': int(tokens[4].replace('sd','')),
                   'epochs': int(tokens[5].replace('ep',''))}
    
    # Update model config
    model_config = parse_model_config(BASE_MODEL_CONFIG)
    model_config = inject_model_config(bdd_set, model_config, hyperparams)
    os.makedirs(os.path.join(TRAINERS_DIRECTORY,trainer, 'cfg'), exist_ok = True)
    model_cfg_path = save_model_config(model_config, os.path.join(TRAINERS_DIRECTORY,trainer, 'cfg', os.path.split(BASE_MODEL_CONFIG)[-1]))
    
    # Update data config
    data_config = parse_data_config(BASE_DATA_CONFIG)
    print(data_config,'\n')
    os.makedirs(os.path.join(TRAINERS_DIRECTORY,trainer, 'data'), exist_ok = True)
    data_config = inject_data_config(bdd_set, data_config)
    print(data_config,'\n')
    data_cfg_path = save_data_config(data_config, os.path.join(TRAINERS_DIRECTORY,trainer, 'cfg', os.path.split(BASE_DATA_CONFIG)[-1]))
    
    ##TODO: Download Darknet 53 weights into backup folder
    
    ## Run Training ##
    TRAINER_DIR = os.path.join(TRAINERS_DIRECTORY,trainer)
    train_results_file = os.path.join(TRAINER_DIR, 'training_results.txt')
    CURRENT_WEIGHT = os.path.join(TRAINERS_DIRECTORY,trainer, 'darknet53.conv.74')
    num_gpus = data_config['gpus']
    darknet_train = "cd {} && ./darknet detector train {} {} {} -gpus {} | tee -a {}".format(WORKING_DIRECTORY,
                                data_cfg_path, model_cfg_path, CURRENT_WEIGHT, num_gpus, train_results_file)
    print('Initializing Training with the following parameters:','\n', darknet_train)
    #res = os.system(darknet_train)      
    #res = subprocess.call([darknet_train])
    proc=Popen(darknet_train, shell=True, stdout=PIPE)

    with open(train_results_file+'.backup',"w+") as f:
        f.write(proc.communicate()[0].decode("utf-8"))

Idx 0
Idx 1
Idx 2
Idx 3
Idx 4
Idx 5
Idx 6
Idx 7
Idx 8
Idx 9
Idx 10
Idx 11
Idx 12
Idx 13
Idx 14
Idx 15
Idx 16
Idx 17
Idx 18
Idx 19
Idx 20
Idx 21
Idx 22
Idx 23
Idx 24
Idx 25
Idx 26
Idx 27
Idx 28
Idx 29
Idx 30
Idx 31
Idx 32
Idx 33
Idx 34
Idx 35
Idx 36
Idx 37
Idx 38
Idx 39
Idx 40
Idx 41
Idx 42
Idx 43
Idx 44
Idx 45
Idx 46
Idx 47
Idx 48
Idx 49
Idx 50
Idx 51
Idx 52
Idx 53
Idx 54
Idx 55
Idx 56
Idx 57
Idx 58
Idx 59
Idx 60
Idx 61
Idx 62
Idx 63
Idx 64
Idx 65
Idx 66
Idx 67
Idx 68
Idx 69
Idx 70
Idx 71
Idx 72
Idx 73
Idx 74
Idx 75
Idx 76
Idx 77
Idx 78
Idx 79
Idx 80
Idx 81
Idx 82
Idx 83
Idx 84
Idx 85
Idx 86
Idx 87
Idx 88
Idx 89
Idx 90
Idx 91
Idx 92
Idx 93
Idx 94
Idx 95
Idx 96
Idx 97
Idx 98
Idx 99
Idx 100
Idx 101
Idx 102
Idx 103
Idx 104
Idx 105
Idx 106
Idx 107
Idx 108
Idx 109
Idx 110
Idx 111
Idx 112
Idx 113
Idx 114
Idx 115
Idx 116
Idx 117
Idx 118
Idx 119
Idx 120
Idx 121
Idx 122
Idx 123
Idx 124
Idx 125
Idx 126
Idx 127
Idx 128
Idx 129
Idx 130
Idx 131
Idx 132
Idx 133
Idx 134
Idx 135
Idx 136
Idx 137
Idx 13

Idx 1126
Idx 1127
Idx 1128
Idx 1129
Idx 1130
Idx 1131
Idx 1132
Idx 1133
Idx 1134
Idx 1135
Idx 1136
Idx 1137
Idx 1138
Idx 1139
Idx 1140
Idx 1141
Idx 1142
Idx 1143
Idx 1144
Idx 1145
Idx 1146
Idx 1147
Idx 1148
Idx 1149
Idx 1150
Idx 1151
Idx 1152
Idx 1153
Idx 1154
Idx 1155
Idx 1156
Idx 1157
Idx 1158
Idx 1159
Idx 1160
Idx 1161
Idx 1162
Idx 1163
Idx 1164
Idx 1165
Idx 1166
Idx 1167
Idx 1168
Idx 1169
Idx 1170
Idx 1171
Idx 1172
Idx 1173
Idx 1174
Idx 1175
Idx 1176
Idx 1177
Idx 1178
Idx 1179
Idx 1180
Idx 1181
Idx 1182
Idx 1183
Idx 1184
Idx 1185
Idx 1186
Idx 1187
Idx 1188
Idx 1189
Idx 1190
Idx 1191
Idx 1192
Idx 1193
Idx 1194
Idx 1195
Idx 1196
Idx 1197
Idx 1198
Idx 1199
Idx 1200
Idx 1201
Idx 1202
Idx 1203
Idx 1204
Idx 1205
Idx 1206
Idx 1207
Idx 1208
Idx 1209
Idx 1210
Idx 1211
Idx 1212
Idx 1213
Idx 1214
Idx 1215
Idx 1216
Idx 1217
Idx 1218
Idx 1219
Idx 1220
Idx 1221
Idx 1222
Idx 1223
Idx 1224
Idx 1225
Idx 1226
Idx 1227
Idx 1228
Idx 1229
Idx 1230
Idx 1231
Idx 1232
Idx 1233
Idx 1234
Idx 1235
Idx 1236
I

Idx 2155
Idx 2156
Idx 2157
Idx 2158
Idx 2159
Idx 2160
Idx 2161
Idx 2162
Idx 2163
Idx 2164
Idx 2165
Idx 2166
Idx 2167
Idx 2168
Idx 2169
Idx 2170
Idx 2171
Idx 2172
Idx 2173
Idx 2174
Idx 2175
Idx 2176
Idx 2177
Idx 2178
Idx 2179
Idx 2180
Idx 2181
Idx 2182
Idx 2183
Idx 2184
Idx 2185
Idx 2186
Idx 2187
Idx 2188
Idx 2189
Idx 2190
Idx 2191
Idx 2192
Idx 2193
Idx 2194
Idx 2195
Idx 2196
Idx 2197
Idx 2198
Idx 2199
Idx 2200
Idx 2201
Idx 2202
Idx 2203
Idx 2204
Idx 2205
Idx 2206
Idx 2207
Idx 2208
Idx 2209
Idx 2210
Idx 2211
Idx 2212
Idx 2213
Idx 2214
Idx 2215
Idx 2216
Idx 2217
Idx 2218
Idx 2219
Idx 2220
Idx 2221
Idx 2222
Idx 2223
Idx 2224
Idx 2225
Idx 2226
Idx 2227
Idx 2228
Idx 2229
Idx 2230
Idx 2231
Idx 2232
Idx 2233
Idx 2234
Idx 2235
Idx 2236
Idx 2237
Idx 2238
Idx 2239
Idx 2240
Idx 2241
Idx 2242
Idx 2243
Idx 2244
Idx 2245
Idx 2246
Idx 2247
Idx 2248
Idx 2249
Idx 2250
Idx 2251
Idx 2252
Idx 2253
Idx 2254
Idx 2255
Idx 2256
Idx 2257
Idx 2258
Idx 2259
Idx 2260
Idx 2261
Idx 2262
Idx 2263
Idx 2264
Idx 2265
I

Idx 3110
Idx 3111
Idx 3112
Idx 3113
Idx 3114
Idx 3115
Idx 3116
Idx 3117
Idx 3118
Idx 3119
Idx 3120
Idx 3121
Idx 3122
Idx 3123
Idx 3124
Idx 3125
Idx 3126
Idx 3127
Idx 3128
Idx 3129
Idx 3130
Idx 3131
Idx 3132
Idx 3133
Idx 3134
Idx 3135
Idx 3136
Idx 3137
Idx 3138
Idx 3139
Idx 3140
Idx 3141
Idx 3142
Idx 3143
Idx 3144
Idx 3145
Idx 3146
Idx 3147
Idx 3148
Idx 3149
Idx 3150
Idx 3151
Idx 3152
Idx 3153
Idx 3154
Idx 3155
Idx 3156
Idx 3157
Idx 3158
Idx 3159
Idx 3160
Idx 3161
Idx 3162
Idx 3163
Idx 3164
Idx 3165
Idx 3166
Idx 3167
Idx 3168
Idx 3169
Idx 3170
Idx 3171
Idx 3172
Idx 3173
Idx 3174
Idx 3175
Idx 3176
Idx 3177
Idx 3178
Idx 3179
Idx 3180
Idx 3181
Idx 3182
Idx 3183
Idx 3184
Idx 3185
Idx 3186
Idx 3187
Idx 3188
Idx 3189
Idx 3190
Idx 3191
Idx 3192
Idx 3193
Idx 3194
Idx 3195
Idx 3196
Idx 3197
Idx 3198
Idx 3199
Idx 3200
Idx 3201
Idx 3202
Idx 3203
Idx 3204
Idx 3205
Idx 3206
Idx 3207
Idx 3208
Idx 3209
Idx 3210
Idx 3211
Idx 3212
Idx 3213
Idx 3214
Idx 3215
Idx 3216
Idx 3217
Idx 3218
Idx 3219
Idx 3220
I

Idx 4146
Idx 4147
Idx 4148
Idx 4149
Idx 4150
Idx 4151
Idx 4152
Idx 4153
Idx 4154
Idx 4155
Idx 4156
Idx 4157
Idx 4158
Idx 4159
Idx 4160
Idx 4161
Idx 4162
Idx 4163
Idx 4164
Idx 4165
Idx 4166
Idx 4167
Idx 4168
Idx 4169
Idx 4170
Idx 4171
Idx 4172
Idx 4173
Idx 4174
Idx 4175
Idx 4176
Idx 4177
Idx 4178
Idx 4179
Idx 4180
Idx 4181
Idx 4182
Idx 4183
Idx 4184
Idx 4185
Idx 4186
Idx 4187
Idx 4188
Idx 4189
Idx 4190
Idx 4191
Idx 4192
Idx 4193
Idx 4194
Idx 4195
Idx 4196
Idx 4197
Idx 4198
Idx 4199
Idx 4200
Idx 4201
Idx 4202
Idx 4203
Idx 4204
Idx 4205
Idx 4206
Idx 4207
Idx 4208
Idx 4209
Idx 4210
Idx 4211
Idx 4212
Idx 4213
Idx 4214
Idx 4215
Idx 4216
Idx 4217
Idx 4218
Idx 4219
Idx 4220
Idx 4221
Idx 4222
Idx 4223
Idx 4224
Idx 4225
Idx 4226
Idx 4227
Idx 4228
Idx 4229
Idx 4230
Idx 4231
Idx 4232
Idx 4233
Idx 4234
Idx 4235
Idx 4236
Idx 4237
Idx 4238
Idx 4239
Idx 4240
Idx 4241
Idx 4242
Idx 4243
Idx 4244
Idx 4245
Idx 4246
Idx 4247
Idx 4248
Idx 4249
Idx 4250
Idx 4251
Idx 4252
Idx 4253
Idx 4254
Idx 4255
Idx 4256
I

Idx 5135
Idx 5136
Idx 5137
Idx 5138
Idx 5139
Idx 5140
Idx 5141
Idx 5142
Idx 5143
Idx 5144
Idx 5145
Idx 5146
Idx 5147
Idx 5148
Idx 5149
Idx 5150
Idx 5151
Idx 5152
Idx 5153
Idx 5154
Idx 5155
Idx 5156
Idx 5157
Idx 5158
Idx 5159
Idx 5160
Idx 5161
Idx 5162
Idx 5163
Idx 5164
Idx 5165
Idx 5166
Idx 5167
Idx 5168
Idx 5169
Idx 5170
Idx 5171
Idx 5172
Idx 5173
Idx 5174
Idx 5175
Idx 5176
Idx 5177
Idx 5178
Idx 5179
Idx 5180
Idx 5181
Idx 5182
Idx 5183
Idx 5184
Idx 5185
Idx 5186
Idx 5187
Idx 5188
Idx 5189
Idx 5190
Idx 5191
Idx 5192
Idx 5193
Idx 5194
Idx 5195
Idx 5196
Idx 5197
Idx 5198
Idx 5199
Idx 5200
Idx 5201
Idx 5202
Idx 5203
Idx 5204
Idx 5205
Idx 5206
Idx 5207
Idx 5208
Idx 5209
Idx 5210
Idx 5211
Idx 5212
Idx 5213
Idx 5214
Idx 5215
Idx 5216
Idx 5217
Idx 5218
Idx 5219
Idx 5220
Idx 5221
Idx 5222
Idx 5223
Idx 5224
Idx 5225
Idx 5226
Idx 5227
Idx 5228
Idx 5229
Idx 5230
Idx 5231
Idx 5232
Idx 5233
Idx 5234
Idx 5235
Idx 5236
Idx 5237
Idx 5238
Idx 5239
Idx 5240
Idx 5241
Idx 5242
Idx 5243
Idx 5244
Idx 5245
I

Idx 6172
Idx 6173
Idx 6174
Idx 6175
Idx 6176
Idx 6177
Idx 6178
Idx 6179
Idx 6180
Idx 6181
Idx 6182
Idx 6183
Idx 6184
Idx 6185
Idx 6186
Idx 6187
Idx 6188
Idx 6189
Idx 6190
Idx 6191
Idx 6192
Idx 6193
Idx 6194
Idx 6195
Idx 6196
Idx 6197
Idx 6198
Idx 6199
Idx 6200
Idx 6201
Idx 6202
Idx 6203
Idx 6204
Idx 6205
Idx 6206
Idx 6207
Idx 6208
Idx 6209
Idx 6210
Idx 6211
Idx 6212
Idx 6213
Idx 6214
Idx 6215
Idx 6216
Idx 6217
Idx 6218
Idx 6219
Idx 6220
Idx 6221
Idx 6222
Idx 6223
Idx 6224
Idx 6225
Idx 6226
Idx 6227
Idx 6228
Idx 6229
Idx 6230
Idx 6231
Idx 6232
Idx 6233
Idx 6234
Idx 6235
Idx 6236
Idx 6237
Idx 6238
Idx 6239
Idx 6240
Idx 6241
Idx 6242
Idx 6243
Idx 6244
Idx 6245
Idx 6246
Idx 6247
Idx 6248
Idx 6249
Idx 6250
Idx 6251
Idx 6252
Idx 6253
Idx 6254
Idx 6255
Idx 6256
Idx 6257
Idx 6258
Idx 6259
Idx 6260
Idx 6261
Idx 6262
Idx 6263
Idx 6264
Idx 6265
Idx 6266
Idx 6267
Idx 6268
Idx 6269
Idx 6270
Idx 6271
Idx 6272
Idx 6273
Idx 6274
Idx 6275
Idx 6276
Idx 6277
Idx 6278
Idx 6279
Idx 6280
Idx 6281
Idx 6282
I

Idx 7196
Idx 7197
Idx 7198
Idx 7199
Idx 7200
Idx 7201
Idx 7202
Idx 7203
Idx 7204
Idx 7205
Idx 7206
Idx 7207
Idx 7208
Idx 7209
Idx 7210
Idx 7211
Idx 7212
Idx 7213
Idx 7214
Idx 7215
Idx 7216
Idx 7217
Idx 7218
Idx 7219
Idx 7220
Idx 7221
Idx 7222
Idx 7223
Idx 7224
Idx 7225
Idx 7226
Idx 7227
Idx 7228
Idx 7229
Idx 7230
Idx 7231
Idx 7232
Idx 7233
Idx 7234
Idx 7235
Idx 7236
Idx 7237
Idx 7238
Idx 7239
Idx 7240
Idx 7241
Idx 7242
Idx 7243
Idx 7244
Idx 7245
Idx 7246
Idx 7247
Idx 7248
Idx 7249
Idx 7250
Idx 7251
Idx 7252
Idx 7253
Idx 7254
Idx 7255
Idx 7256
Idx 7257
Idx 7258
Idx 7259
Idx 7260
Idx 7261
Idx 7262
Idx 7263
Idx 7264
Idx 7265
Idx 7266
Idx 7267
Idx 7268
Idx 7269
Idx 7270
Idx 7271
Idx 7272
Idx 7273
Idx 7274
Idx 7275
Idx 7276
Idx 7277
Idx 7278
Idx 7279
Idx 7280
Idx 7281
Idx 7282
Idx 7283
Idx 7284
Idx 7285
Idx 7286
Idx 7287
Idx 7288
Idx 7289
Idx 7290
Idx 7291
Idx 7292
Idx 7293
Idx 7294
Idx 7295
Idx 7296
Idx 7297
Idx 7298
Idx 7299
Idx 7300
Idx 7301
Idx 7302
Idx 7303
Idx 7304
Idx 7305
Idx 7306
I

Idx 8185
Idx 8186
Idx 8187
Idx 8188
Idx 8189
Idx 8190
Idx 8191
Idx 8192
Idx 8193
Idx 8194
Idx 8195
Idx 8196
Idx 8197
Idx 8198
Idx 8199
Idx 8200
Idx 8201
Idx 8202
Idx 8203
Idx 8204
Idx 8205
Idx 8206
Idx 8207
Idx 8208
Idx 8209
Idx 8210
Idx 8211
Idx 8212
Idx 8213
Idx 8214
Idx 8215
Idx 8216
Idx 8217
Idx 8218
Idx 8219
Idx 8220
Idx 8221
Idx 8222
Idx 8223
Idx 8224
Idx 8225
Idx 8226
Idx 8227
Idx 8228
Idx 8229
Idx 8230
Idx 8231
Idx 8232
Idx 8233
Idx 8234
Idx 8235
Idx 8236
Idx 8237
Idx 8238
Idx 8239
Idx 8240
Idx 8241
Idx 8242
Idx 8243
Idx 8244
Idx 8245
Idx 8246
Idx 8247
Idx 8248
Idx 8249
Idx 8250
Idx 8251
Idx 8252
Idx 8253
Idx 8254
Idx 8255
Idx 8256
Idx 8257
Idx 8258
Idx 8259
Idx 8260
Idx 8261
Idx 8262
Idx 8263
Idx 8264
Idx 8265
Idx 8266
Idx 8267
Idx 8268
Idx 8269
Idx 8270
Idx 8271
Idx 8272
Idx 8273
Idx 8274
Idx 8275
Idx 8276
Idx 8277
Idx 8278
Idx 8279
Idx 8280
Idx 8281
Idx 8282
Idx 8283
Idx 8284
Idx 8285
Idx 8286
Idx 8287
Idx 8288
Idx 8289
Idx 8290
Idx 8291
Idx 8292
Idx 8293
Idx 8294
Idx 8295
I

Idx 9238
Idx 9239
Idx 9240
Idx 9241
Idx 9242
Idx 9243
Idx 9244
Idx 9245
Idx 9246
Idx 9247
Idx 9248
Idx 9249
Idx 9250
Idx 9251
Idx 9252
Idx 9253
Idx 9254
Idx 9255
Idx 9256
Idx 9257
Idx 9258
Idx 9259
Idx 9260
Idx 9261
Idx 9262
Idx 9263
Idx 9264
Idx 9265
Idx 9266
Idx 9267
Idx 9268
Idx 9269
Idx 9270
Idx 9271
Idx 9272
Idx 9273
Idx 9274
Idx 9275
Idx 9276
Idx 9277
Idx 9278
Idx 9279
Idx 9280
Idx 9281
Idx 9282
Idx 9283
Idx 9284
Idx 9285
Idx 9286
Idx 9287
Idx 9288
Idx 9289
Idx 9290
Idx 9291
Idx 9292
Idx 9293
Idx 9294
Idx 9295
Idx 9296
Idx 9297
Idx 9298
Idx 9299
Idx 9300
Idx 9301
Idx 9302
Idx 9303
Idx 9304
Idx 9305
Idx 9306
Idx 9307
Idx 9308
Idx 9309
Idx 9310
Idx 9311
Idx 9312
Idx 9313
Idx 9314
Idx 9315
Idx 9316
Idx 9317
Idx 9318
Idx 9319
Idx 9320
Idx 9321
Idx 9322
Idx 9323
Idx 9324
Idx 9325
Idx 9326
Idx 9327
Idx 9328
Idx 9329
Idx 9330
Idx 9331
Idx 9332
Idx 9333
Idx 9334
Idx 9335
Idx 9336
Idx 9337
Idx 9338
Idx 9339
Idx 9340
Idx 9341
Idx 9342
Idx 9343
Idx 9344
Idx 9345
Idx 9346
Idx 9347
Idx 9348
I

In [ ]:
bdd_set.export(datasets.Format.scalabel)